In [1]:
!pip install xgboost
!pip install tensorflow

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder


2025-03-10 07:14:48.418415: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741590888.433783    9978 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741590888.438383    9978 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-10 07:14:48.456937: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Step 1: Load the dataset
df = pd.read_csv(r"/workspaces/codespaces-jupyter/data/covtype.csv")  # Change file name if needed

le = LabelEncoder()
df.iloc[:, -1] = le.fit_transform(df.iloc[:, -1])  # Assuming the last column is your target

In [4]:
# Step 2: Preprocessing
y = df.iloc[:, -1]  # Target variable (Cover Type)
X = df.iloc[:, :-1]  # Features



In [5]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Step 3: Train individual models
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
xgb_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
lgbm_model = LGBMClassifier()

In [8]:
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:16:26] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.068411 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2148
[LightGBM] [Info] Number of data points in the train set: 464809, number of used features: 53
[LightGBM] [Info] Start training from score -1.010055
[LightGBM] [Info] Start training from score -0.717554
[LightGBM] [Info] Start training from score -2.787067
[LightGBM] [Info] Start training from score -5.343669
[LightGBM] [Info] Start training from score -4.126990
[LightGBM] [Info] Start training from score -3.511322
[LightGBM] [Info] Start training from score -3.338569


LGBMClassifier()

In [11]:
# Step 4: Make Predictions
y_pred_rf = rf_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)
y_pred_lgbm = lgbm_model.predict(X_test)

y_pred_rf
y_pred_xgb
y_pred_lgbm

array([0, 1, 1, ..., 1, 1, 6])

In [12]:
# Step 5: Ensemble Model
ensemble_model = VotingClassifier(
    estimators=[('rf', rf_model), ('xgb', xgb_model), ('lgbm', lgbm_model)], voting='hard')
ensemble_model.fit(X_train, y_train)
y_pred_ensemble = ensemble_model.predict(X_test)

/usr/local/python/3.10.13/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [07:19:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.054824 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2148
[LightGBM] [Info] Number of data points in the train set: 464809, number of used features: 53
[LightGBM] [Info] Start training from score -1.010055
[LightGBM] [Info] Start training from score -0.717554
[LightGBM] [Info] Start training from score -2.787067
[LightGBM] [Info] Start training from score -5.343669
[LightGBM] [Info] Start training from score -4.126990
[LightGBM] [Info] Start training from score -3.511322
[LightGBM] [Info] Start training from score -3.338569


In [13]:
# Step 6: Neural Network
nn_model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(len(y.unique()), activation='softmax')
])

/usr/local/python/3.10.13/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-03-10 07:20:30.051853: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
nn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
nn_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

Epoch 1/5


2025-03-10 07:20:54.068155: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 80318952 exceeds 10% of free system memory.


11621/11621 ━━━━━━━━━━━━━━━━━━━━ 17s 1ms/step - accuracy: 0.7430 - loss: 0.5982 - val_accuracy: 0.7803 - val_loss: 0.5033
Epoch 2/5
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.7666 - loss: 0.5470 - val_accuracy: 0.7953 - val_loss: 0.4832
Epoch 3/5
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.7743 - loss: 0.5276 - val_accuracy: 0.8120 - val_loss: 0.4577
Epoch 4/5
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.7837 - loss: 0.5091 - val_accuracy: 0.8144 - val_loss: 0.4386
Epoch 5/5
11621/11621 ━━━━━━━━━━━━━━━━━━━━ 16s 1ms/step - accuracy: 0.7879 - loss: 0.4999 - val_accuracy: 0.8250 - val_loss: 0.4279


In [16]:
# Step 7: Compare Accuracy
nn_pred = np.argmax(nn_model.predict(X_test), axis=1)

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred_xgb))
print("LightGBM Accuracy:", accuracy_score(y_test, y_pred_lgbm))
print("Ensemble Model Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print("Neural Network Accuracy:", accuracy_score(y_test, nn_pred))

3632/3632 ━━━━━━━━━━━━━━━━━━━━ 2s 634us/step
Random Forest Accuracy: 0.9551388518368716
XGBoost Accuracy: 0.8711823274786366
LightGBM Accuracy: 0.8485839436159135
Ensemble Model Accuracy: 0.8860958839272652
Neural Network Accuracy: 0.8238255466726332
